In [1]:
!pip install mediapipe

     ---------------------------------------- 49.8/49.8 MB 6.4 MB/s eta 0:00:00
     -------------------------------------- 124.6/124.6 kB 3.6 MB/s eta 0:00:00
     -------------------------------------- 904.2/904.2 kB 9.5 MB/s eta 0:00:00


In [1]:
import cv2
import numpy as np
import mediapipe as mp
  
# Configuration Face Mesh.
mp_face_mesh = mp.solutions.face_mesh
face_mesh =  mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
  
img = cv2.imread('320654582_483194113959822_3066599250046978556_n.jpg', cv2.IMREAD_UNCHANGED)
image = cv2.cvtColor(cv2.flip(img, 1), cv2.COLOR_BGR2RGB)
# To improve performance.
image.flags.writeable = False
results =  face_mesh.process(image)

In [2]:
(results.multi_face_landmarks[0].landmark[205].x * image.shape[1], results.multi_face_landmarks[0].landmark[205].y * image.shape[0])


(499.45507049560547, 198.82525420188904)

In [13]:
!pip install faceBlendCommon

ERROR: Could not find a version that satisfies the requirement faceBlendCommon (from versions: none)
ERROR: No matching distribution found for faceBlendCommon


In [5]:
!pip install faceBlendCommon


ERROR: Could not find a version that satisfies the requirement faceBlendCommon (from versions: none)
ERROR: No matching distribution found for faceBlendCommon


In [8]:
!python faceBlendCommon.py install

In [9]:
import mediapipe as mp
import cv2
import math
import numpy as np
import faceBlendCommon as fbc
import csv

In [22]:
!pip install load_filter_img

ERROR: Could not find a version that satisfies the requirement load_filter_img (from versions: none)
ERROR: No matching distribution found for load_filter_img


In [11]:
VISUALIZE_FACE_POINTS = False
 
filters_config = {
    'anonymous':
        [{'path': "filters/anonymous.png",
         'anno_path': "filters/anonymous_annotations.csv",
         'morph': True, 'animated': False, 'has_alpha': True}],
    'anime':
        [{'path': "filters/anime.png",
         'anno_path': "filters/anime_annotations.csv",
         'morph': True, 'animated': False, 'has_alpha': True}],
    'dog':
        [{'path': "filters/dog-ears.png",
         'anno_path': "filters/dog-ears_annotations.csv",
         'morph': False, 'animated': False, 'has_alpha': True},
         {'path': "filters/dog-nose.png",
          'anno_path': "filters/dog-nose_annotations.csv",
          'morph': False, 'animated': False, 'has_alpha': True}],
    'cat':
        [{'path': "filters/cat-ears.png",
         'anno_path': "filters/cat-ears_annotations.csv",
         'morph': False, 'animated': False, 'has_alpha': True},
         {'path': "filters/cat-nose.png",
          'anno_path': "filters/cat-nose_annotations.csv",
          'morph': False, 'animated': False, 'has_alpha': True}],
}

In [12]:
def getLandmarks(img):
    mp_face_mesh = mp.solutions.face_mesh
    selected_keypoint_indices = [127, 93, 58, 136, 150, 149, 176, 148, 152, 377, 400, 378, 379, 365, 288, 323, 356, 70, 63, 105, 66, 55,
                 285, 296, 334, 293, 300, 168, 6, 195, 4, 64, 60, 94, 290, 439, 33, 160, 158, 173, 153, 144, 398, 385,
                 387, 466, 373, 380, 61, 40, 39, 0, 269, 270, 291, 321, 405, 17, 181, 91, 78, 81, 13, 311, 306, 402, 14,
                 178, 162, 54, 67, 10, 297, 284, 389]
 
    height, width = img.shape[:-1]
 
    with mp_face_mesh.FaceMesh(max_num_faces=1, static_image_mode=True, min_detection_confidence=0.5) as face_mesh:
 
        results = face_mesh.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
 
        if not results.multi_face_landmarks:
            print('Face not detected!!!')
            return 0
 
        for face_landmarks in results.multi_face_landmarks:
            values = np.array(face_landmarks.landmark)
            face_keypnts = np.zeros((len(values), 2))
 
            for idx,value in enumerate(values):
                face_keypnts[idx][0] = value.x
                face_keypnts[idx][1] = value.y
 
            # Convert normalized points to image coordinates
            face_keypnts = face_keypnts * (width, height)
            face_keypnts = face_keypnts.astype('int')
 
            relevant_keypnts = []
 
            for i in selected_keypoint_indices:
                relevant_keypnts.append(face_keypnts[i])
            return relevant_keypnts
    return 0

In [13]:
def load_filter(img_path, has_alpha):
    # Read the image
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
 
    alpha = None
    if has_alpha:
        b, g, r, alpha = cv2.split(img)
        img = cv2.merge((b, g, r))
 
    return img, alpha

In [14]:
def load_landmarks(annotation_file):
    with open(annotation_file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",")
        points = {}
        for i, row in enumerate(csv_reader):
            # skip head or empty line if it's there
            try:
                x, y = int(row[1]), int(row[2])
                points[row[0]] = (x, y)
            except ValueError:
                continue
        return points

In [15]:
def find_convex_hull(points):
    hull = []
    hullIndex = cv2.convexHull(np.array(list(points.values())), clockwise=False, returnPoints=False)
    addPoints = [
        [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59],  # Outer lips
        [60], [61], [62], [63], [64], [65], [66], [67],  # Inner lips
        [27], [28], [29], [30], [31], [32], [33], [34], [35],  # Nose
        [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47],  # Eyes
        [17], [18], [19], [20], [21], [22], [23], [24], [25], [26]  # Eyebrows
    ]
    hullIndex = np.concatenate((hullIndex, addPoints))
    for i in range(0, len(hullIndex)):
        hull.append(points[str(hullIndex[i][0])])
 
    return hull, hullIndex

In [31]:
def load_filter(filter_name="dog"):
 
    filters = filters_config[filter_name]
 
    multi_filter_runtime = []
 
    for filter in filters:
        temp_dict = {}
 
        img1, img1_alpha = load_filter.img(filter['path'], filter['has_alpha'])
 
        temp_dict['img'] = img1
        temp_dict['img_a'] = img1_alpha
 
        points = load_landmarks(filter['anno_path'])
 
        temp_dict['points'] = points
 
        if filter['morph']:
            # Find convex hull for delaunay triangulation using the landmark points
            hull, hullIndex = find_convex_hull(points)
 
            # Find Delaunay triangulation for convex hull points
            sizeImg1 = img1.shape
            rect = (0, 0, sizeImg1[1], sizeImg1[0])
            dt = fbc.calculateDelaunayTriangles(rect, hull)
 
            temp_dict['hull'] = hull
            temp_dict['hullIndex'] = hullIndex
            temp_dict['dt'] = dt
 
            if len(dt) == 0:
                continue
 
        if filter['animated']:
            filter_cap = cv2.VideoCapture(filter['path'])
            temp_dict['cap'] = filter_cap
 
        multi_filter_runtime.append(temp_dict)
 
    return filters, multi_filter_runtime

In [32]:
# Process input from webcam or video file
cap = cv2.VideoCapture(0)
 
# Some variables we will need later
count = 0
isFirstFrame = True
sigma = 50
 
# Load an initial filter
iter_filter_keys = iter(filters_config.keys())
filters, multi_filter_runtime = load_filter(next(iter_filter_keys))

AttributeError: 'function' object has no attribute 'img'